In [1]:
!pip install --upgrade pip
!pip install git+https://github.com/deepset-ai/haystack.git#egg=farm-haystack[colab]

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 22.0.4
    Uninstalling pip-22.0.4:
      Successfully uninstalled pip-22.0.4
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/deepset-ai/haystack.git to /tmp/pip-install-ti55ure9/farm-haystack_c7627e40f5ec430eb05e74191fcd791b
  Running command git clone --filter=blob:none --quiet https://github.com/deepset-ai/haystack.git /tmp/pip-install-ti55ure9/farm-haystack_c7627e40f5ec430eb05e74191fcd791b
  Resolved https://github.com/deepset-ai/haystack.git to commit 4a0a0541645e699036e4b6b18d783cb2c864d4fc
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from haystack.nodes import FARMReader

In [3]:
from haystack.nodes import FARMReader
reader = FARMReader(model_name_or_path="distilbert-base-uncased-distilled-squad", use_gpu=True)


Downloading:   0%|          | 0.00/451 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/265M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
data_dir = "/content/drive/MyDrive/data"

In [5]:
reader.train(data_dir=data_dir, train_filename="dev-v2.0.json", use_gpu=True, n_epochs=1, save_dir="my_model",)

Preprocessing dataset:   0%|          | 0/3 [00:00<?, ? Dicts/s]

  0%|          | 0/1360 [00:00<?, ?it/s]

In [6]:
reader.save(directory="my_model")

In [7]:
new_reader = FARMReader(model_name_or_path="my_model")

In [9]:
reader_eval_results = new_reader.eval_on_file("/content/drive/MyDrive/data", "dev-v2.0.json", device="cuda")

- instead of giving you full control over which labels to use, this method always returns three types of metrics: combined (no suffix), text_answer ('_text_answer' suffix) and no_answer ('_no_answer' suffix) metrics.
- instead of comparing predictions with labels on a string level, this method compares them on a token-ID level. This makes it unable to do any string normalization (e.g. normalize whitespaces) beforehand.
Hence, results might slightly differ from those of `Pipeline.eval()`
.If you are just about starting to evaluate your model consider using `Pipeline.eval()` instead.


Preprocessing dataset:   0%|          | 0/3 [00:00<?, ? Dicts/s]

Evaluating:   0%|          | 0/272 [00:00<?, ?it/s]

In [10]:
reader_eval_results

{'EM': 50.8964316797215,
 'f1': 50.8964316797215,
 'top_n_accuracy': 99.3733681462141,
 'top_n': 4,
 'EM_text_answer': 4.125468803273099,
 'f1_text_answer': 4.125468803273099,
 'top_n_accuracy_text_answer': 98.77258779406752,
 'top_n_EM_text_answer': 93.04466416638255,
 'top_n_f1_text_answer': 97.05531308195164,
 'Total_text_answer': 2933,
 'EM_no_answer': 99.67994310099573,
 'f1_no_answer': 99.67994310099573,
 'top_n_accuracy_no_answer': 100.0,
 'Total_no_answer': 2812}

In [11]:
context = '''Bangalore (/bæŋɡəˈlɔːr/), officially known as Bengaluru (Kannada pronunciation: [ˈbeŋɡəɭuːɾu] (audio speaker iconlisten)), is the capital and the largest city of the Indian state of Karnataka. It has a population of more than 8 million and a metropolitan population of around 11 million, making it the third most populous city and fifth most populous urban agglomeration in India.[12] Located in southern India on the Deccan Plateau, at a height of over 900 m (3,000 ft) above sea level, Bangalore is known for its pleasant climate throughout the year. Its elevation is the highest among the major cities of India.[13]'''


In [12]:
new_reader.predict_on_texts("what is the language spoken in Karnataka?",[context])

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]

{'query': 'what is the language spoken in Karnataka?',
 'no_ans_gap': -3.9583005905151367,
 'answers': [<Answer {'answer': 'Kannada', 'type': 'extractive', 'score': 0.11943735182285309, 'context': 'Bangalore (/bæŋɡəˈlɔːr/), officially known as Bengaluru (Kannada pronunciation: [ˈbeŋɡəɭuːɾu] (audio speaker iconlisten)), is the capital and the larg', 'offsets_in_document': [{'start': 57, 'end': 64}], 'offsets_in_context': [{'start': 57, 'end': 64}], 'document_id': 'f035ef1741f01a771e207677acb66291', 'meta': {}}>]}

In [ ]:
#Inference Using Pipeline

In [13]:
from haystack import Pipeline, Document
from haystack.utils import print_answers
# reader = FARMReader(model_name_or_path="deepset/roberta-base-squad2")
p = Pipeline()
p.add_node(component=new_reader, name="Reader", inputs=["Query"])
res = p.run(
    query="what is the largest city in karnataka? ", documents=[Document(content=context)]
)
print_answers(res,details="medium")

Inferencing Samples:   0%|          | 0/1 [00:00<?, ? Batches/s]


Query: what is the largest city in karnataka? 
Answers:
[   {   'answer': 'Bangalore',
        'context': 'Bangalore (/bæŋɡəˈlɔːr/), officially known as Bengaluru '
                   '(Kannada pronunciation: [ˈbeŋɡəɭuːɾu] (audio speaker '
                   'iconlisten)), is the capital and the larg',
        'score': 0.4019722640514374}]
